# First-Order Data Generator Tutorial

##  Lernziele / Learning Objectives

In diesem Tutorial lernen Sie:
1. Was First-Order Daten sind und warum sie wichtig sind
2. Wie man den `FirstOrderDataGenerator` verwendet
3. Wie man Verteilungen speichert und lädt
4. Wie man mit `FirstOrderDataset` und DataLoader arbeitet
5. Wie man ein Modell mit Soft Targets trainiert

**Voraussetzungen:** Grundkenntnisse in PyTorch

**Dauer:** ~30 Minuten

##  Teil 1: Einführung und Setup

### Was sind First-Order Daten?

In Machine Learning arbeiten wir mit der Verteilung $p(X, Y)$, wobei:
- $X$ = Eingabe-Features
- $Y$ = Ziel-Labels

Die **bedingte Verteilung** $p(Y|X)$ sagt uns: "Gegeben ein bestimmtes $x$, wie wahrscheinlich sind die verschiedenen Klassen?"

**Problem:** Normalerweise haben wir keinen Zugang zu $p(Y|X)$!

**Lösung:** Wir approximieren diese mit einem gut trainierten Modell $\hat{h}$:
$$\hat{h}(x) \approx p(\cdot | x)$$

Diese Approximationen nennen wir **First-Order Daten**.

### Installation und Imports

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# First-Order Generator importieren
# HINWEIS: Passen Sie den Import-Pfad an Ihre Projektstruktur an!
from probly.data_generator.first_order_generator import (
    FirstOrderDataGenerator,
    FirstOrderDataset,
    output_fo_dataloader
)

print(" Alle Imports erfolgreich!")
print(f"PyTorch Version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

##  Teil 2: Beispiel-Daten vorbereiten

Wir erstellen einen einfachen Datensatz und ein "Teacher"-Modell, das wir als Ground Truth verwenden.

In [ ]:
# Beispiel-Dataset
class SimpleDataset(Dataset):
    """Ein einfacher Datensatz für Demonstrations-Zwecke."""
    
    def __init__(self, n_samples: int = 200, input_dim: int = 10, n_classes: int = 3, seed: int = 42) -> None:
        """Initialize dataset."""
        torch.manual_seed(seed)
        self.n_samples = n_samples
        self.input_dim = input_dim
        self.n_classes = n_classes
        
        # Generiere synthetische Daten
        self.data = torch.randn(n_samples, input_dim)
        self.labels = torch.randint(0, n_classes, (n_samples,))
    
    def __len__(self) -> int:
        """Return length."""
        return self.n_samples
    
    def __getitem__(self, idx: int) -> tuple:
        """Get item."""
        return self.data[idx], self.labels[idx]

# Dataset erstellen
dataset = SimpleDataset(n_samples=200, input_dim=10, n_classes=3)
print(f" Dataset erstellt: {len(dataset)} Samples, {dataset.n_classes} Klassen")

# Beispiel-Sample anschauen
sample_x, sample_y = dataset[0]
print(f"\nBeispiel Sample:")
print(f"  Input shape: {sample_x.shape}")
print(f"  Label: {sample_y}")

In [ ]:
# Teacher-Modell (repräsentiert die "Ground Truth")
class TeacherModel(nn.Module):
    """Ein einfaches neuronales Netzwerk als Teacher-Modell."""
    
    def __init__(self, input_dim: int = 10, n_classes: int = 3) -> None:
        """Initialize model."""
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, n_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass."""
        return self.network(x)  # Gibt Logits zurück

# Modell erstellen
teacher_model = TeacherModel(input_dim=10, n_classes=3)
teacher_model.eval()  # Wichtig: In Evaluation-Modus!

print(" Teacher-Modell erstellt")
print(f"\nModell-Architektur:")
print(teacher_model)

##  Teil 3: First-Order Verteilungen generieren

Jetzt verwenden wir den `FirstOrderDataGenerator`, um für jedes Sample im Dataset eine Wahrscheinlichkeitsverteilung zu berechnen.

In [ ]:
# Generator initialisieren
generator = FirstOrderDataGenerator(
    model=teacher_model,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    batch_size=32,
    output_mode='logits',  # Unser Modell gibt Logits aus
    model_name='teacher_v1'
)

print(" FirstOrderDataGenerator initialisiert")
print(f"  Device: {generator.device}")
print(f"  Batch Size: {generator.batch_size}")
print(f"  Output Mode: {generator.output_mode}")

In [ ]:
# Verteilungen generieren
print("⏳ Generiere First-Order Verteilungen...\n")

distributions = generator.generate_distributions(
    dataset,
    progress=True  # Zeigt Fortschritt an
)

print(f"\n {len(distributions)} Verteilungen generiert!")

In [ ]:
# Beispiel-Verteilungen anschauen
print(" Beispiel-Verteilungen:\n")

for i in range(5):
    dist = distributions[i]
    print(f"Sample {i}:")
    print(f"  Verteilung: [{dist[0]:.4f}, {dist[1]:.4f}, {dist[2]:.4f}]")
    print(f"  Summe: {sum(dist):.6f} (sollte ≈ 1.0 sein)")
    print(f"  Wahrscheinlichste Klasse: {np.argmax(dist)}")
    print()

In [ ]:
# Visualisierung: Verteilungen für erste 10 Samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    ax = axes[i]
    dist = distributions[i]
    
    ax.bar(range(len(dist)), dist, color=['blue', 'orange', 'green'])
    ax.set_title(f'Sample {i}')
    ax.set_xlabel('Klasse')
    ax.set_ylabel('Wahrscheinlichkeit')
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.suptitle('First-Order Verteilungen für die ersten 10 Samples', y=1.02, fontsize=14)
plt.show()

print(" Visualisierung: Jeder Balken zeigt die Wahrscheinlichkeit für eine Klasse")

##  Teil 4: Verteilungen speichern und laden

Wir können die generierten Verteilungen als JSON-Datei speichern, um sie später wiederzuverwenden.

In [ ]:
# Verzeichnis erstellen
output_dir = Path('tutorial_output')
output_dir.mkdir(exist_ok=True)

# Pfad definieren
save_path = output_dir / 'first_order_distributions.json'

# Metadaten definieren
metadata = {
    'dataset': 'SimpleDataset',
    'n_samples': len(dataset),
    'n_classes': dataset.n_classes,
    'input_dim': dataset.input_dim,
    'note': 'Generated for tutorial purposes',
    'teacher_architecture': 'Simple 3-layer network'
}

# Speichern
print(f" Speichere Verteilungen nach: {save_path}")
generator.save_distributions(
    path=save_path,
    distributions=distributions,
    meta=metadata
)
print(" Erfolgreich gespeichert!")

# Dateigröße anzeigen
file_size = save_path.stat().st_size / 1024  # in KB
print(f"\n Dateigröße: {file_size:.2f} KB")

In [ ]:
# Verteilungen laden
print(" Lade Verteilungen...\n")

loaded_distributions, loaded_metadata = generator.load_distributions(save_path)

print(" Erfolgreich geladen!\n")
print(" Metadaten:")
for key, value in loaded_metadata.items():
    print(f"  - {key}: {value}")

# Verifizierung
print(f"\n Verifizierung:")
print(f"  Anzahl Verteilungen: {len(loaded_distributions)}")
print(f"  Daten identisch: {distributions == loaded_distributions}")

##  Teil 5: FirstOrderDataset verwenden

`FirstOrderDataset` ist ein PyTorch Dataset-Wrapper, der den ursprünglichen Datensatz mit den First-Order Verteilungen kombiniert.

In [ ]:
# FirstOrderDataset erstellen
fo_dataset = FirstOrderDataset(
    base_dataset=dataset,
    distributions=loaded_distributions
)

print(f" FirstOrderDataset erstellt mit {len(fo_dataset)} Samples\n")

# Ein Sample abrufen
input_tensor, label, distribution = fo_dataset[0]

print(" Sample 0:")
print(f"  Input Shape: {input_tensor.shape}")
print(f"  Label: {label}")
print(f"  Distribution Shape: {distribution.shape}")
print(f"  Distribution: [{distribution[0]:.4f}, {distribution[1]:.4f}, {distribution[2]:.4f}]")
print(f"  Summe: {distribution.sum():.6f}")

In [ ]:
# Mehrere Samples durchgehen
print(" Durchlaufe mehrere Samples:\n")

for i in range(5):
    input_tensor, label, distribution = fo_dataset[i]
    predicted_class = torch.argmax(distribution).item()
    confidence = distribution[predicted_class].item()
    
    print(f"Sample {i}:")
    print(f"  Ground Truth Label: {label}")
    print(f"  Predicted Class: {predicted_class}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Match: {'' if predicted_class == label else ''}")
    print()

##  Teil 6: DataLoader erstellen

Für das Training brauchen wir einen DataLoader, der Batches mit First-Order Verteilungen liefert.

In [ ]:
# DataLoader mit First-Order Verteilungen erstellen
fo_loader = output_fo_dataloader(
    base_dataset=dataset,
    distributions=loaded_distributions,
    batch_size=32,
    shuffle=True,
    num_workers=0  # Für Windows-Kompatibilität
)

print(f" DataLoader erstellt")
print(f"  Batch Size: 32")
print(f"  Anzahl Batches: {len(fo_loader)}")
print(f"  Shuffle: True")

In [ ]:
# Ersten Batch anschauen
batch_inputs, batch_labels, batch_distributions = next(iter(fo_loader))

print("\n Erster Batch:")
print(f"  Inputs Shape: {batch_inputs.shape}")
print(f"  Labels Shape: {batch_labels.shape}")
print(f"  Distributions Shape: {batch_distributions.shape}")
print(f"\n  Erste 3 Verteilungen im Batch:")
for i in range(3):
    dist = batch_distributions[i]
    print(f"    Sample {i}: [{dist[0]:.4f}, {dist[1]:.4f}, {dist[2]:.4f}]")

##  Teil 7: Student-Modell mit Soft Targets trainieren

Jetzt trainieren wir ein "Student"-Modell, das versucht, die Verteilungen des Teacher-Modells zu lernen. Dies nennt man **Knowledge Distillation**.

In [ ]:
# Student-Modell (kleineres Netzwerk)
class StudentModel(nn.Module):
    """Ein kleineres Modell, das vom Teacher lernt."""
    
    def __init__(self, input_dim: int = 10, n_classes: int = 3) -> None:
        """Initialize model."""
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 32),  # Kleiner als Teacher
            nn.ReLU(),
            nn.Linear(32, n_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass."""
        return self.network(x)

# Student-Modell erstellen
student_model = StudentModel(input_dim=10, n_classes=3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_model = student_model.to(device)

print(" Student-Modell erstellt")
print(f"\nVergleich Teacher vs Student:")
print(f"  Teacher Parameter: {sum(p.numel() for p in teacher_model.parameters())}")
print(f"  Student Parameter: {sum(p.numel() for p in student_model.parameters())}")
    print(f"  Student ist {ratio:.1f}x kleiner!")


In [ ]:
# Training-Setup
optimizer = torch.optim.Adam(student_model.parameters(), lr=0.001)
epochs = 10

# Listen für Tracking
train_losses = []

print(" Starte Training...\n")

for epoch in range(epochs):
    student_model.train()
    epoch_loss = 0.0
    n_batches = 0
    
    for inputs, _labels, target_distributions in fo_loader:
        # Daten zum Device verschieben
        batch_inputs = inputs.to(device)
        batch_target_distributions = target_distributions.to(device)
        
        # Forward pass
        logits = student_model(batch_inputs)
        
        # KL Divergence Loss
        # Student versucht, die Teacher-Verteilungen zu imitieren
        log_probs = F.log_softmax(logits, dim=-1)
        loss = F.kl_div(
            log_probs,
            target_distributions,
            reduction='batchmean'
        )
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    avg_loss = epoch_loss / n_batches
    train_losses.append(avg_loss)
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

print("\n Training abgeschlossen!")

In [ ]:
# Visualisierung: Training Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs+1), train_losses, marker='o', linewidth=2, markersize=8)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('KL Divergence Loss', fontsize=12)
plt.title('Training Loss über Epochen', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n Loss-Reduktion: {train_losses[0]:.4f} → {train_losses[-1]:.4f}")
print(f"   Verbesserung: {(1 - train_losses[-1]/train_losses[0])*100:.1f}%")

##  Teil 8: Evaluation - Teacher vs Student

Vergleichen wir die Vorhersagen des Teacher-Modells mit denen des trainierten Student-Modells.

In [ ]:
# Evaluation-Modus
student_model.eval()
teacher_model.eval()

# Vorhersagen sammeln
all_inputs = []
teacher_probs_list = []
student_probs_list = []
true_labels = []

with torch.no_grad():
    for i in range(len(dataset)):
        x, y = dataset[i]
        x = x.unsqueeze(0).to(device)  # Batch dimension
        
        # Teacher predictions
        teacher_logits = teacher_model(x)
        teacher_probs = F.softmax(teacher_logits, dim=-1)
        
        # Student predictions
        student_logits = student_model(x)
        student_probs = F.softmax(student_logits, dim=-1)
        
        all_inputs.append(x.cpu())
        teacher_probs_list.append(teacher_probs.cpu())
        student_probs_list.append(student_probs.cpu())
        true_labels.append(y)

# Zu Tensoren konvertieren
teacher_probs_all = torch.cat(teacher_probs_list, dim=0)
student_probs_all = torch.cat(student_probs_list, dim=0)
true_labels_all = torch.tensor(true_labels)

print(" Evaluation abgeschlossen")
print(f"\n Evaluiert auf {len(dataset)} Samples")

In [ ]:
# Accuracy berechnen
teacher_predictions = torch.argmax(teacher_probs_all, dim=-1)
student_predictions = torch.argmax(student_probs_all, dim=-1)

teacher_accuracy = (teacher_predictions == true_labels_all).float().mean().item()
student_accuracy = (student_predictions == true_labels_all).float().mean().item()

print(" Accuracy:")
print(f"  Teacher: {teacher_accuracy*100:.2f}%")
print(f"  Student: {student_accuracy*100:.2f}%")
print(f"\n  Differenz: {abs(teacher_accuracy - student_accuracy)*100:.2f} Prozentpunkte")

In [ ]:
# KL Divergence zwischen Teacher und Student berechnen
kl_div = F.kl_div(
    F.log_softmax(student_probs_all, dim=-1),
    teacher_probs_all,
    reduction='batchmean'
).item()

print(f" Durchschnittliche KL Divergence zwischen Teacher und Student: {kl_div:.4f}")
print(f"\n Interpretation:")
print(f"   Niedriger Wert ({kl_div:.4f}) bedeutet, dass der Student die")
print(f"   Teacher-Verteilungen gut gelernt hat!")

In [ ]:
# Visualisierung: Teacher vs Student für einige Samples
n_samples_to_show = 6
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i in range(n_samples_to_show):
    ax = axes[i]
    
    teacher_dist = teacher_probs_all[i].numpy()
    student_dist = student_probs_all[i].numpy()
    true_label = true_labels_all[i].item()
    
    x = np.arange(len(teacher_dist))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, teacher_dist, width, label='Teacher', alpha=0.8)
    bars2 = ax.bar(x + width/2, student_dist, width, label='Student', alpha=0.8)
    
    # Markiere true label
    ax.axvline(x=true_label, color='red', linestyle='--', linewidth=2, label='True Label')
    
    ax.set_title(f'Sample {i} (True Label: {true_label})', fontweight='bold')
    ax.set_xlabel('Klasse')
    ax.set_ylabel('Wahrscheinlichkeit')
    ax.set_ylim([0, 1])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.suptitle('Vergleich: Teacher vs Student Predictions', y=1.02, fontsize=16, fontweight='bold')
plt.show()

print("\n Die Balken zeigen, wie ähnlich die Student-Predictions den Teacher-Predictions sind")

##  Teil 9: Coverage-Metrik berechnen

Coverage misst, wie gut die Student-Verteilungen die Teacher-Verteilungen "abdecken".

In [ ]:
def compute_coverage(pred_probs: torch.Tensor, target_probs: torch.Tensor, epsilon: float = 0.15) -> float:
    """
    Berechnet epsilon-Credal Coverage.
    
    Eine Vorhersage "deckt" das Target ab, wenn die L1-Distanz <= epsilon ist.
    """
    l1_distance = torch.sum(torch.abs(pred_probs - target_probs), dim=-1)
    covered = (l1_distance <= epsilon).float()
    return covered.mean().item()

# Coverage für verschiedene Epsilon-Werte
epsilons = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
coverages = []

for eps in epsilons:
    cov = compute_coverage(student_probs_all, teacher_probs_all, epsilon=eps)
    coverages.append(cov)
    print(f"Coverage bei ε = {eps:.2f}: {cov*100:.2f}%")

In [ ]:
# Visualisierung: Coverage vs Epsilon
plt.figure(figsize=(10, 6))
plt.plot(epsilons, [c*100 for c in coverages], marker='o', linewidth=2, markersize=10)
plt.xlabel('Epsilon (ε)', fontsize=12)
plt.ylabel('Coverage (%)', fontsize=12)
plt.title('Coverage in Abhängigkeit von Epsilon', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.ylim([0, 105])

# Markiere optimalen Punkt
optimal_idx = len(coverages) // 2
plt.axvline(x=epsilons[optimal_idx], color='red', linestyle='--', alpha=0.5, label='Beispiel ε')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n Interpretation:")
print(f"   Je höher das Epsilon, desto mehr Vorhersagen werden als 'covered' gezählt.")
print(f"   Ein gutes Modell hat hohe Coverage bei kleinem Epsilon!")

##  Teil 10: Erweiterte Analysen

Schauen wir uns an, bei welchen Samples der Student am besten und am schlechtesten abschneidet.

In [ ]:
# L1-Distanzen für alle Samples berechnen
l1_distances = torch.sum(torch.abs(student_probs_all - teacher_probs_all), dim=-1)

# Beste und schlechteste Samples finden
best_indices = torch.argsort(l1_distances)[:5]  # 5 beste
worst_indices = torch.argsort(l1_distances, descending=True)[:5]  # 5 schlechteste

print(" Top 5 Samples (kleinste L1-Distanz):")
for i, idx in enumerate(best_indices):
    idx = idx.item()
    dist = l1_distances[idx].item()
    print(f"  {i+1}. Sample {idx}: L1-Distanz = {dist:.4f}")

print("\n  Bottom 5 Samples (größte L1-Distanz):")
for i, idx in enumerate(worst_indices):
    idx = idx.item()
    dist = l1_distances[idx].item()
    print(f"  {i+1}. Sample {idx}: L1-Distanz = {dist:.4f}")

In [ ]:
# Histogramm der L1-Distanzen
plt.figure(figsize=(10, 6))
plt.hist(l1_distances.numpy(), bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('L1-Distanz', fontsize=12)
plt.ylabel('Anzahl Samples', fontsize=12)
plt.title('Verteilung der L1-Distanzen zwischen Teacher und Student', fontsize=14, fontweight='bold')
plt.axvline(x=l1_distances.mean().item(), color='red', linestyle='--', linewidth=2, label=f'Mittelwert: {l1_distances.mean().item():.3f}')
plt.axvline(x=l1_distances.median().item(), color='green', linestyle='--', linewidth=2, label=f'Median: {l1_distances.median().item():.3f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n Statistiken der L1-Distanzen:")
print(f"   Mittelwert: {l1_distances.mean().item():.4f}")
print(f"   Median: {l1_distances.median().item():.4f}")
print(f"   Standardabweichung: {l1_distances.std().item():.4f}")
print(f"   Min: {l1_distances.min().item():.4f}")
print(f"   Max: {l1_distances.max().item():.4f}")

##  Teil 11: Zusammenfassung und Best Practices

### Was haben wir gelernt?

1. **First-Order Daten** sind Approximationen der bedingten Verteilung $p(Y|X)$
2. Der **FirstOrderDataGenerator** macht es einfach, diese zu generieren
3. Verteilungen können **gespeichert und geladen** werden (JSON-Format)
4. **FirstOrderDataset** kombiniert Daten mit Verteilungen
5. **Knowledge Distillation** nutzt First-Order Daten als Soft Targets
6. **Coverage** ist eine wichtige Metrik für Unsicherheitsquantifizierung

### Best Practices

 **DO:**
- Modell immer in `eval()` Modus setzen vor Generierung
- Metadaten beim Speichern hinzufügen
- `shuffle=False` beim Generieren verwenden
- Verteilungen regelmäßig verifizieren (Summe = 1.0)

 **DON'T:**
- Modell nicht im Training-Modus lassen
- Index-Alignment nicht ignorieren
- Nicht ohne Metadaten speichern
- Gerätekonsistenz nicht vergessen

### Nächste Schritte

1. Probieren Sie es mit Ihren eigenen Modellen und Datensätzen
2. Experimentieren Sie mit verschiedenen `output_mode` Einstellungen
3. Implementieren Sie benutzerdefinierte `input_getter` Funktionen
4. Erkunden Sie erweiterte Anwendungsfälle (z.B. Credal Sets)
5. Vergleichen Sie verschiedene Teacher-Modelle

##  Übungsaufgaben

Versuchen Sie folgende Erweiterungen:

1. **Experiment 1**: Ändern Sie die Teacher-Architektur und beobachten Sie die Auswirkungen auf Coverage
2. **Experiment 2**: Verwenden Sie verschiedene Temperaturen beim Softmax (`F.softmax(logits/T, dim=-1)`)
3. **Experiment 3**: Implementieren Sie einen Ensemble-Ansatz mit mehreren Teacher-Modellen
4. **Experiment 4**: Visualisieren Sie die Konfidenz-Kalibrierung
5. **Experiment 5**: Testen Sie mit einem echten Datensatz (z.B. MNIST oder CIFAR-10)

##  Weitere Ressourcen

- **Dokumentation**: `docs/data_generation_guide.md`
- **API-Referenz**: `docs/api_reference.md`
- **Beispiel-Skript**: `examples/simple_usage.py`
- **Tests**: `tests/test_first_order_generator.py`

### Literatur

- Hinton et al. (2015): "Distilling the Knowledge in a Neural Network"
- Guo et al. (2017): "On Calibration of Modern Neural Networks"
- Lakshminarayanan et al. (2017): "Simple and Scalable Predictive Uncertainty Estimation using Deep Ensembles"

---

**Tutorial erstellt von**: ProblyPros Team  
**Version**: 1.0  
**Letzte Aktualisierung**: Dezember 2024

Bei Fragen oder Feedback: Erstellen Sie ein Issue im Repository! 